<a href="https://colab.research.google.com/github/anubhavgupta1/Dive-Into-Deep-Learning/blob/main/Linear%20Regression/Concise%20Implementation/mxnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

In [1]:
!pip install d2l==0.16.1
!pip install -U mxnet-cu101==1.7.0

In [2]:
from d2l import mxnet as d2l
from mxnet import autograd, gluon, np, npx
npx.set_np()

### Generating the Dataset

In [3]:
true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### Reading the Dataset

In [4]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a Gluon data iterator."""
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [6]:
next(iter(data_iter))

[array([[ 0.55241627,  0.21463516],
        [-0.20801921, -0.54877394],
        [-0.71534044, -0.45822975],
        [ 0.34593368, -0.7567092 ],
        [-0.05871473,  0.60052294],
        [-0.55203336, -0.6159344 ],
        [ 0.43054336,  0.8985137 ],
        [ 0.62162733,  0.3694969 ],
        [ 0.24499686, -1.6080191 ],
        [-0.15916686,  0.8726678 ]]), array([[ 4.603484 ],
        [ 5.656826 ],
        [ 4.3228054],
        [ 7.478742 ],
        [ 2.034631 ],
        [ 5.2043667],
        [ 1.9865699],
        [ 4.1955047],
        [10.171097 ],
        [ 0.9134456]])]

### Defining the Model

In [7]:
# `nn` is an abbreviation for neural networks
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

###Initializing Model Parameters

In [8]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

### Defining the Loss Function

In [9]:
loss = gluon.loss.L2Loss()

###Defining the Optimization Algorithm

In [10]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

### Training

In [11]:
num_epochs = 50
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    if(epoch==0 or epoch%10==9):
      print(f'epoch {epoch + 1}, loss {l.mean().asnumpy():f}')

epoch 1, loss 0.024870
epoch 10, loss 0.000051
epoch 20, loss 0.000051
epoch 30, loss 0.000051
epoch 40, loss 0.000051
epoch 50, loss 0.000051


In [12]:
w = net[0].weight.data()
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
b = net[0].bias.data()
print(f'error in estimating b: {true_b - b}')

error in estimating w: [3.081560e-04 2.002716e-05]
error in estimating b: [0.00027561]
